# Let's cook model

Let's combine what we've found so far.

- [What are ingredients?](https://www.kaggle.com/rejasupotaro/what-are-ingredients) (Preprocessing & Feature extraction)
- [Representations for ingredients](https://www.kaggle.com/rejasupotaro/representations-for-ingredients)

Steps are below.

1. Load dataset
2. Remove outliers
3. Preprocess
4. Create model
5. Check local CV
6. Train model
7. Check predicted values
8. Make submission

In [ ]:
import json
import re
import unidecode
import numpy as np
import pandas as pd
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_validate
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from tqdm import tqdm
tqdm.pandas()

## 1. Load dataset

In [ ]:
train = pd.read_json('../input/train.json')
test = pd.read_json('../input/test.json')

## 2. Remove outliers

I saw weird recipes in the dataset .

- water => Japanese
- butter => Indian
- butter => French

Let's filter such single-ingredient recipes and see how it goes.

In [ ]:
train['num_ingredients'] = train['ingredients'].apply(len)
train = train[train['num_ingredients'] > 1]

## 3. Preprocess

Currently, the preprocess is like below.

- convert to lowercase
- remove hyphen
- remove numbers
- remove words which consist of less than 2 characters
- lemmatize

This process can be better.

In [ ]:
lemmatizer = WordNetLemmatizer()
def preprocess(ingredients):
    ingredients_text = ' '.join(ingredients)
    ingredients_text = ingredients_text.lower()
    ingredients_text = ingredients_text.replace('-', ' ')
    words = []
    for word in ingredients_text.split():
        if re.findall('[0-9]', word): continue
        if len(word) <= 2: continue
        if '’' in word: continue
        word = lemmatizer.lemmatize(word)
        if len(word) > 0: words.append(word)
    return ' '.join(words)

for ingredient, expected in [
    ('Eggs', 'egg'),
    ('all-purpose flour', 'all purpose flour'),
    ('purée', 'purée'),
    ('1% low-fat milk', 'low fat milk'),
    ('half & half', 'half half'),
    ('safetida (powder)', 'safetida (powder)')
]:
    actual = preprocess([ingredient])
    assert actual == expected, f'"{expected}" is excpected but got "{actual}"'

In [ ]:
train['x'] = train['ingredients'].progress_apply(preprocess)
test['x'] = test['ingredients'].progress_apply(preprocess)
train.head()

I need to tune the parameters of TfidfVectorizer later.

In [ ]:
vectorizer = make_pipeline(
    TfidfVectorizer(sublinear_tf=True),
    FunctionTransformer(lambda x: x.astype('float16'), validate=False)
)

x_train = vectorizer.fit_transform(train['x'].values)
x_train.sort_indices()
x_test = vectorizer.transform(test['x'].values)

Encode cuisines to numeric values using LabelEncoder.

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train['cuisine'].values)
dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

## 4. Create model

I've tried LogisticRegression, GaussianProcessClassifier, GradientBoostingClassifier, MLPClassifier, LGBMClassifier, SGDClassifier, Keras but SVC works better so far.

I need to take a look at models and the parameters more closely.

In [ ]:
estimator = SVC(
    C=80,
    kernel='rbf',
    gamma=1.7,
    coef0=1,
    cache_size=500,
)
classifier = OneVsRestClassifier(estimator, n_jobs=-1)

## 5. Check local CV

TRUST YOUR LOCAL CV. TRUST YOUR LOCAL CV. TRUST YOUR LOCAL CV. I repeated 3 times since this is the most important thing.

Try different prprocesses and parameters while looking at the local CV.

In [ ]:
%%time
scores = cross_validate(classifier, x_train, y_train, cv=3)
scores['test_score'].mean()

## 6. Train model

If I become to be confident in the model, I train it with the whole train data for submission.

In [ ]:
%%time
classifier.fit(x_train, y_train)

## 7. Check predicted values

Check if the model fitted enough.

In [ ]:
y_pred = label_encoder.inverse_transform(classifier.predict(x_train))
y_true = label_encoder.inverse_transform(y_train)

print(f'accuracy score on train data: {accuracy_score(y_true, y_pred)}')

def report2dict(cr):
    rows = []
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0: rows.append(parsed_row)
    measures = rows[0]
    classes = defaultdict(dict)
    for row in rows[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            classes[class_label][m.strip()] = float(row[j + 1].strip())
    return classes
report = classification_report(y_true, y_pred)
pd.DataFrame(report2dict(report)).T

## 6. Make submission

It seems to be working well. Let's make a submission.

In [ ]:
y_pred = label_encoder.inverse_transform(classifier.predict(x_test))
test['cuisine'] = y_pred
test[['id', 'cuisine']].to_csv('submission.csv', index=False)
test[['id', 'cuisine']].head()

That's it! Don't trust what I've done here. The score can be better. Please let me know if you find a better approach.